In [14]:
from lasertram import LaserTRAM, LaserCalc
import lasertram as lt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm


In [15]:
data = pd.read_excel(r"C:\Users\jlubbers\OneDrive - DOI\Research\Coding\laserTRAM-DB\tests\spot_test_timestamp_raw_data.xlsx").set_index('SampleLabel')
data.head()

,timestamp,Time,7Li,24Mg,27Al,29Si,43Ca,48Ti,57Fe,88Sr,138Ba,139La,140Ce,153Eu,208Pb
SampleLabel,,,,,,,,,,,,,,,
BCR-2G_1,2021-03-01 22:08:14,13.24,500.010000,0.0,100.0004,46285.535670,100.0004,0.0000,600.014400,0.0,0.0,0.0,0.0,0.0,100.0004
BCR-2G_1,2021-03-01 22:08:14,163.60,700.019601,0.0,200.0016,43274.778818,0.0000,200.0016,900.032401,0.0,0.0,0.0,0.0,0.0,0.0000
BCR-2G_1,2021-03-01 22:08:14,313.96,600.014400,0.0,400.0064,46586.651171,300.0036,200.0016,1000.040002,0.0,0.0,0.0,0.0,0.0,200.0016
BCR-2G_1,2021-03-01 22:08:14,464.23,800.025601,0.0,100.0004,45683.326387,100.0004,100.0004,1400.078404,0.0,0.0,0.0,0.0,0.0,0.0000
BCR-2G_1,2021-03-01 22:08:14,614.50,600.014400,0.0,300.0036,46185.165445,200.0016,200.0016,700.019601,0.0,0.0,0.0,0.0,0.0,0.0000


In [16]:
samples = data.index.unique().dropna().tolist()

sample = samples[0]

bkgd_interval = (5,10)
keep_interval = (20,50)

spot = LaserTRAM(name = sample)
spot.get_data(data.loc[sample,:])
spot.assign_int_std("29Si")
spot.assign_intervals(bkgd_interval, keep_interval)
spot.get_bkgd_data()
spot.subtract_bkgd()
spot.get_detection_limits()
spot.normalize_interval()
spot.make_output_report()

In [17]:
spot.output_report

,timestamp,Spot,despiked,bkgd_start,bkgd_stop,int_start,int_stop,norm,norm_cps,7Li,...,29Si_se,43Ca_se,48Ti_se,57Fe_se,88Sr_se,138Ba_se,139La_se,140Ce_se,153Eu_se,208Pb_se
0,2021-03-01 22:08:14,BCR-2G_1,None,5.12408,10.08472,20.00647,50.07226,29Si,2499024.199695,0.001831,...,0.0,6.061414,3.21829,3.000996,1.807116,4.168718,3.689592,3.973801,18.846732,16.604926


In [18]:
spot = LaserTRAM(name = sample)
lt.process_spot(
        spot,
        raw_data = data.loc[sample,:],
        bkgd = (5,10),
        keep = (20,50),
        internal_std = "29Si",
        despike = False,
        output_report = True
        )

In [19]:
bkgd_interval = (5, 8)
keep_interval = (25, 45)

int_std = "29Si"

my_spots = []
print("PROCESSING SPOTS\n")
for sample in tqdm(samples):
    spot = LaserTRAM(name=sample)
    lt.process_spot(
        spot,
        raw_data=data.loc[sample, :],
        bkgd=bkgd_interval,
        keep=keep_interval,
        internal_std=int_std,
        despike=False,
        output_report=True,
    )
    my_spots.append(spot)

print("COMBINING OUTPUT REPORTS\n")
processed_df = pd.DataFrame()
for spot in tqdm(my_spots):
    processed_df = pd.concat([processed_df, spot.output_report])

PROCESSING SPOTS



  0%|          | 0/40 [00:00<?, ?it/s]

COMBINING OUTPUT REPORTS



  0%|          | 0/40 [00:00<?, ?it/s]

In [20]:
processed_df.head()

,timestamp,Spot,despiked,bkgd_start,bkgd_stop,int_start,int_stop,norm,norm_cps,7Li,...,29Si_se,43Ca_se,48Ti_se,57Fe_se,88Sr_se,138Ba_se,139La_se,140Ce_se,153Eu_se,208Pb_se
0,2021-03-01 22:08:14,BCR-2G_1,None,5.12408,8.13056,25.11728,45.11153,29Si,2495372.560199,0.001819,...,0.0,0.850964,0.692265,0.682130,0.707998,0.587471,0.992761,0.845244,1.635309,1.329348
0,2021-03-01 22:09:31,BCR-2G_2,None,5.12417,8.1308,25.11798,45.1115,29Si,2546833.925726,0.001797,...,0.0,0.637530,0.541900,0.497523,0.568544,0.626896,0.598107,0.591970,1.444278,0.865801
0,2021-03-01 22:10:48,ATHO-G_1,None,5.12431,8.13076,25.11669,45.11084,29Si,2530573.062027,0.003787,...,0.0,1.321670,0.467695,0.466671,0.498083,0.508516,0.476264,0.536568,1.509867,6.488146
0,2021-03-01 22:12:05,ATHO-G_2,None,5.12421,8.1309,25.11759,45.1118,29Si,2595616.459733,0.003875,...,0.0,0.829131,0.365662,0.364831,0.423951,0.359817,0.423304,0.393812,1.247436,1.064285
0,2021-03-01 22:13:22,BHVO-2G_1,None,5.12455,8.13149,25.11798,45.11006,29Si,1812510.243358,0.000907,...,0.0,1.093567,1.223828,1.280427,1.207772,1.257159,0.970521,1.854826,1.657233,11.591823


In [21]:
srm_data = pd.read_excel(r"C:\Users\jlubbers\OneDrive - DOI\Research\Coding\laserTRAM-DB\tests\laicpms_stds_tidy.xlsx")


In [22]:
concentrations = LaserCalc(name="experiment_test")
concentrations.get_SRM_comps(srm_data)
concentrations.get_data(processed_df)
concentrations.set_calibration_standard("BCR-2G")
concentrations.drift_check()
concentrations.get_calibration_std_ratios()
concentrations.set_internal_standard_concentrations(
    concentrations.data["Spot"],
    np.full(concentrations.data["Spot"].shape[0], 71.9),
    np.full(concentrations.data["Spot"].shape[0], 1),
)
concentrations.calculate_concentrations()
concentrations.get_secondary_standard_accuracies()



In [24]:
concentrations.SRM_accuracies.head()

,timestamp,Spot,7Li,24Mg,27Al,29Si,43Ca,48Ti,57Fe,88Sr,138Ba,139La,140Ce,153Eu,208Pb
sample,,,,,,,,,,,,,,,
ATHO-G,2021-03-01 22:10:48,ATHO-G_1,90.842068,93.443173,102.119695,100.0,102.036726,100.309370,93.279587,102.500955,100.901892,105.281488,101.963682,100.817969,103.536497
ATHO-G,2021-03-01 22:12:05,ATHO-G_2,92.871338,92.767870,102.273820,100.0,100.895524,101.654184,94.007586,103.402998,100.599368,104.099827,102.279146,102.844627,100.726420
ATHO-G,2021-03-01 22:39:01,ATHO-G_3,91.367009,95.587086,103.349109,100.0,104.635706,103.866733,95.745150,106.448856,103.477759,108.453288,105.708050,104.560567,102.675767
ATHO-G,2021-03-01 23:00:51,ATHO-G_4,92.506019,94.901562,103.161392,100.0,101.617686,103.570813,96.287984,105.925031,104.182972,107.420743,104.650498,105.616961,102.817465
ATHO-G,2021-03-01 23:22:43,ATHO-G_5,93.861539,96.930322,104.274223,100.0,104.930349,105.398300,96.280831,107.398161,105.687959,109.135171,106.601549,105.907334,107.224674
